In [1]:
%load_ext autoreload
%autoreload 2

from filemanager import fileset
from cli import cli

fileset_manager = fileset.PDSFileset()

qgis_logs = r'Z:\ABP56\Log&Track\Filesets'
filesets_folder = r'Z:\ABP56\Log&Track\Filesets'

In [2]:
# pick qgis log
qgis_log_list = cli.search(qgis_logs, '.csv')

Searching *.csv files in directory:Z:\ABP56\Log&Track\Filesets
Scanned filesets are: 

0 Z:\ABP56\Log&Track\Filesets\abp56_mbes_fstrack_seb01.csv
1 Z:\ABP56\Log&Track\Filesets\abp56_mbes_fstrack_seb02.csv
2 Z:\ABP56\Log&Track\Filesets\abp56_mbes_fstrack_seb03.csv


In [3]:
qgis_log_picked = cli.pick_from_list(qgis_log_list)

Please type two numbers that define the range of filesets you like to copy or move
Please type the first number: 


Please type the second number: 
Do you want to add more elements? (Y/N)


Picked elements:
abp56_mbes_fstrack_seb03.csv


In [4]:
import shutil
import os

def qgislogToFilesets(output_dir, qgis_log):
    
    filesets = []
    svp_filesets = []
    filesets_dict = {}

    with open(qgis_log, 'r') as csv_file:
        file_content = csv_file.read().splitlines()

        header = file_content[0].split(',')
        
        filename_idx = 0
        svp_idx = 0
        fileset_idx = 0
        print(header)
        print(filename_idx)
        for field in header:
            if field == 'Fname' or field == 'fname':
                break
            else:
                filename_idx += 1
        for field in header:
            if field == 'svp_statio':
                break
            else:
                svp_idx += 1
                
        for field in header:
            if field == 'fileset':
                break
            else:
                fileset_idx += 1
    
        for num, line in enumerate(file_content[1:]):
            line_content = line.split(',')
            file_name = line_content[filename_idx]
            fileset_name = line_content[fileset_idx]
            svp_name = line_content[svp_idx]

            if fileset_name in filesets_dict.keys():
                filesets_dict[fileset_name].append((file_name, svp_name))
            else:
                filesets_dict[fileset_name] = []
                filesets_dict[fileset_name].append((file_name, svp_name))
    
    for key in filesets_dict.keys():
        fileset_name = key

        fs = fileset.PDSFileset()
        fs.setname(fileset_name)
        
        for filename, svp_name in filesets_dict[key]:
            fs.addfile(filename=filename, svp_name=svp_name)
        filesets.append(fs)
        svp_fs = fs.create_svp_filesets()
        svp_filesets.append(svp_fs)
    
    
    for fs, svpfs in zip(filesets,svp_filesets):
        fs_dir = os.path.join(output_dir,fs.name)
        
        if not os.path.exists(fs_dir):
            os.mkdir(fs_dir)
            
        print(f'{fs.name} - {len(fs.files)}')
        fs.savefileset(dirpath=fs_dir)
        len_svp = 0
        for svfs in svpfs:
            print(f'{svfs.name} - {len(svfs.files)}')
            svfs.savefileset(dirpath=fs_dir)
            len_svp += len(svfs.files)
        print(len_svp)

In [5]:
qgislogToFilesets(filesets_folder, qgis_log_picked[0])

['fid', 'fname', 'begin', 'end', 'uid', 'fname_2', 'tsmp', 'datetime', 'x', 'y', 'svp_statio', 'fname_svps', 'fileset', 'proc_by', 'status']
0
abp56_id009_RybSouth_Tr_utm34n - 44
abp56_id009_RybSouth_Tr_utm34n_ABP56001 - 36
abp56_id009_RybSouth_Tr_utm34n_ABP56002 - 8
44
abp56_id006_SambWest_Tr_utm34n - 29
abp56_id006_SambWest_Tr_utm34n_ABP56001 - 29
29
abp56_id003_DuneDynamics2_Po_utm34n - 85
abp56_id003_DuneDynamics2_Po_utm34n_ABP56001 - 81
abp56_id003_DuneDynamics2_Po_utm34n_ABP56002 - 4
85
abp56_id004_Channel_Tr_utm34n - 160
abp56_id004_Channel_Tr_utm34n_ABP56001 - 160
160
abp56_id005_SambWest_Tr_utm34n - 26
abp56_id005_SambWest_Tr_utm34n_ABP56001 - 26
26
abp56_id001_Dune01_Po_utm34n - 140
abp56_id001_Dune01_Po_utm34n_ABP56003 - 105
abp56_id001_Dune01_Po_utm34n_ABP56002 - 35
140
abp56_id010_RybSouth_Tr_utm34n - 27
abp56_id010_RybSouth_Tr_utm34n_ABP56003 - 27
27
abp56_id002_Dune01_Po_utm34n - 138
abp56_id002_Dune01_Po_utm34n_ABP56002 - 110
abp56_id002_Dune01_Po_utm34n_ABP56003 - 23
a